<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Import_CV2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 07/19/2021-3
from __future__ import absolute_import
from IPython.display import clear_output

import glob, os, shutil
from os.path import *

try: 
    from google.colab import drive, files
    from google.colab.patches import cv2_imshow
except Exception as err:
    import google
    from google.colab.patches import cv2_imshow

try: 
    from matplotlib import pyplot as plt
except Exception as err:
    %pip install matplotlib
    from matplotlib import pyplot as plt

try: 
    import cv2
    from cv2 import COLOR_BGR2RGBA
except Exception as err:
    print(err)
    %pip install opencv-python-headless # works.
    # %pip install opencv-contrib-python-headless
    import cv2
    from cv2 import COLOR_BGR2RGBA, IMREAD_UNCHANGED

import numpy as np

if exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

contentPth = os.getcwd()
tensorflowImagesPath = join(contentPth, 'TensorflowImages')
resizedImagesPath = join(contentPth, 'TensorflowResizedImages')

if exists(tensorflowImagesPath):
    pass
    # shutil.rmtree(tensorflowImagesPath)

try:
    drive.flush_and_unmount()
    drive.mount('/content/gdrive/', force_remount=True)
    mydrivePth = '/content/gdrive/MyDrive'
    os.chdir(mydrivePth)
    gdriveTFImagesPth = join(mydrivePth, 'TensorflowImages')
    gdriveResizedImagesPth = join(mydrivePth, 'TensorflowResizedImages')
    
    os.chdir(mydrivePth)
    from TarfileFunctions import *
    from BashColors import C
    
    if not exists(tensorflowImagesPath):
        shutil.copytree(gdriveResizedImagesPth, tensorflowImagesPath)
        # tff.listTarfiles()
        os.chdir(contentPth)
        # tff.extractTarfiles('Juno_Original_Images.tar.gz')

    gdrive_resized_images = '/content/gdrive/MyDrive/TensorflowResizedImages'
    if not exists(resizedImagesPath): # and not exists(resizedImagesPath):
        print('from:', gdrive_resized_images)
        print('to:', resizedImagesPath)
        try: shutil.copytree(gdrive_resized_images, resizedImagesPath)
        except Exception as err:
            print(err)

    os.chdir(contentPth)

except Exception as err:
    # print(err)
    os.chdir(contentPth)
    from TarfileFunctions import *
    from BashColors import C
    tarfilePath=join(contentPth, 'Juno_Original_Images.tar.gz')
    if exists(tarfilePath) and not exists(tensorflowImagesPath):
        tff.extractTarfiles('Juno_Original_Images.tar.gz')

rootPth = expanduser("~")

os.chdir(contentPth)
print(f'cv2: {C.IBlue}{cv2.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}')

In [ ]:
os.chdir(contentPth)

directoryList=[]
fileList=[]
dirCount=0
fileCount=0
globList = glob.glob('**', recursive=True)
for fil in sorted(globList):
    fullPath = abspath(fil)
    if isdir(fullPath):
        dirCount+=1
        print(f'\n{C.IBlue}{fullPath}{C.ColorOff}')
        directoryList.append(fullPath)
    elif isfile(fullPath) and fullPath.endswith('.png'):
        fileCount+=1
        print(fileCount, fullPath)
        fileList.append(fullPath)
clear_output()

In [ ]:
#  Create backgroundImg.png
backgroundImg = np.zeros(shape=(224,224,4))
backgroundImgSavePath = join('/content/gdrive/MyDrive/TensorflowResizedImages',
                             'backgroundImage224.png')
bgImg = cv2.imwrite(backgroundImgSavePath, backgroundImg)
bgImg = cv2.imread(backgroundImgSavePath, cv2.IMREAD_UNCHANGED)

cv2.waitKey(1)
cv2.destroyAllWindows()

for px in bgImg:
    print(px.ndim, px.shape, px.strides)
print(basename(backgroundImgSavePath))
cv2_imshow(bgImg)
cv2.waitKey(2500)
bgImgZeroPixel = bgImg[0][0]
# print(bgImgZeroPixel, bgImg.shape)
clear_output()

In [ ]:
os.chdir(resizedImagesPath)
imageGlob= glob.glob('**', recursive=True)
imageGlobPathList=[]
count = 0
for path in sorted(imageGlob):
    path=abspath(path)
    if path.endswith('.png'):
        imageGlobPathList.append(path)
        count += 1
        print(f'{count}. {basename(path)}')
        # print(path)
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)

        cv2_imshow(img)
        cv2.waitKey(2500)
        print(img.shape)
        cv2.destroyAllWindows()

imageGlobPathList=sorted(imageGlobPathList)


In [ ]:

print(len(imageGlobPathList), 'files')
count=0
for path in sorted(imageGlobPathList):
    count+=1
    imagePath = abspath(path)
    # print(imagePath)
    imageName = basename(imagePath)
    
    img = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
    # print(type(img))
    # print('original image:',img.shape)
    
    zoomImg_H = cv2.hconcat([bgImg, img, bgImg])
    bgImg_H = cv2.hconcat([bgImg, bgImg, bgImg])
    
    newImg = cv2.vconcat([bgImg_H, zoomImg_H, bgImg_H])
    newImg = cv2.resize(newImg, (224, 224), cv2.INTER_CUBIC)
    
    imageName = 'zoom_' + imageName
    saveImagePath = join(contentPth, imageName)
    # print('saveImagePath:', saveImagePath)
    
    cv2.imwrite(saveImagePath, newImg)
    newImg = cv2.imread(saveImagePath, cv2.IMREAD_UNCHANGED)

    print(f'\n{count}. {imageName}')
    cv2_imshow(newImg)
    cv2.waitKey(1500)
    cv2.destroyAllWindows()
    print('ndim:', newImg.ndim, 'shape:', newImg.shape, 'size:', newImg.size)

In [ ]:
q
cv2_imshow(img)
plt.imshow(img)
bgPixel = img[1][1]
print(bgPixel.shape)
count1=0
count2=0
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        thisPixel = img[i][j]

        if thisPixel.all() == bgPixel.all():
            count1+=1

        if thisPixel[0] == 64:
            thisPixel[0] = 0

        elif thisPixel[1] == 64:
            count1+=1
            thisPixel[1] = 0

        elif thisPixel[2] == 64:
            thisPixel[2] = 0

        if thisPixel.all() == bgPixel.all():
            count2+=1
            # print(a[3])
        else: pass

        #print(img[i][j])
        
print('count1:', count1)
print('count2:', count2)
print('img.size:', img.size)
print(thisPixel.shape)

In [ ]:
import cv2
b,g,r,a = cv2.split(img)
t=(b,g,r,a)

newImg = cv2.merge([b,g,r,a])
plt.imshow(newImg)

for px in newImg:
    pass

newImgPath = join(contentPth, 'newEnemy.png')
cv2.imwrite(
    filename=newImgPath,
    img=newImg
)

try: 
    gdriveNewImgPath = join(mydrivePth, 'newEnemy.png')
    cv2.imwrite(gdriveNewImgPath, newImg)
except: pass



In [ ]:


imgZeroPixel=img[0][0]
bgCount=0
pxCount=0
print(f'bgPixel: {imgZeroPixel}')
plt.imshow(img)